In [65]:
import pandas as pd
import numpy as np
import os
import math
import re

In [5]:
data = pd.read_excel('../data/raw/nyc_benchmarking_disclosure_2017_consumption_data.xlsx', sheet_name='Information and Metrics')

In [8]:
data_columns = data.columns

In [13]:
data

,Order,Property Id,Property Name,Parent Property Id,Parent Property Name,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Address 2 (self-reported),...,Annual Maximum Demand (kW),Annual Maximum Demand (MM/YYYY),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Water Use (All Water Sources) (kgal),Water Use Intensity (All Water Sources) (gal/ft²),Water Required?,Generation Date,DOF Benchmarking Submission Status
0,1,4593574,The Argonaut Building,NaN,NaN,1.010288e+09,1010287502,1024898,224 West 57th St,NaN,...,NaN,NaT,732.4,76.3,656.1,3635.5,21.46,Not found,2018-02-14,Not found
1,3,2967701,Cathedral Preparatory Seminary,NaN,NaN,4.018720e+09,4-01872-0007,4046340,56-25 92nd Street,NaN,...,NaN,NaT,164.5,109.9,54.6,102.9,1.09,Not found,2018-02-14,Not found
2,4,4898531,The Nomad Hotel,NaN,NaN,1.008290e+09,1-00829-0050,1080710,1170 Broadway,NaN,...,NaN,NaT,1150.2,438.0,712.3,10762.6,86.10,Not found,2018-02-14,Not found
3,5,2917939,10 West 27 Street Corp,NaN,NaN,1.008280e+09,1-00828-0053,1015657,1155 Broadway,NaN,...,NaN,NaT,273.4,98.2,175.3,790.1,15.80,Not found,2018-02-14,Not found
4,6,3878205,Westbury Realty,NaN,NaN,1.000650e+09,1-00065-0024,1001105,24 John Street,NaN,...,NaN,NaT,90.7,0.0,90.7,143.0,2.86,Not found,2018-02-14,Not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34350,34682,6682473,Mitchell Hill- 2802 Clarendon Road,NaN,NaN,3.051940e+09,3-05194-0001,3119966,2802 Clarendon Road,NaN,...,NaN,NaT,187.1,160.2,26.8,295851.5,9019.86,Not found,2019-03-07,Not found
34351,34683,6682474,Mitchell Hill- 1116 Carroll Street,NaN,NaN,3.012880e+09,3-01288-0038,3033818,1116 Carroll Street,NaN,...,NaN,NaT,181.2,157.7,23.5,283262.5,8756.18,Not found,2019-03-07,Not found
34352,34684,6682477,Mitchell Hill- 3033 Brighton 13 Street,NaN,NaN,3.087160e+09,3-08716-0085,3245653,3033 Brighton 13 Street,NaN,...,NaN,NaT,230.4,177.0,53.3,451795.0,13366.71,Not found,2019-03-07,Not found
34353,34685,2819133,The Ritz Tower - 465 Park Avenue,NaN,NaN,1.013128e+09,1-01312-7502,1036922,465 Park Avenue,NaN,...,NaN,NaT,614.8,23.7,591.1,7118.2,27.04,Not found,2019-03-08,Not found


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34355 entries, 0 to 34354
Data columns (total 60 columns):
 #   Column                                                      Non-Null Count  Dtype         
---  ------                                                      --------------  -----         
 0   Order                                                       34355 non-null  int64         
 1   Property Id                                                 34355 non-null  int64         
 2   Property Name                                               34355 non-null  object        
 3   Parent Property Id                                          1723 non-null   object        
 4   Parent Property Name                                        1723 non-null   object        
 5   BBL - 10 digits                                             33625 non-null  float64       
 6   NYC Borough, Block and Lot (BBL) self-reported              33626 non-null  object        
 7   NYC Building Identific

In [14]:
# drop colums if they contain more than 70% NAs
data_cleaned = data.dropna(thresh=int(0.7*len(data)),axis=1)

In [23]:
data_cleaned.drop(columns=['Order', 'Property Id'], inplace=True)

KeyError: "['Order', 'Property Id'] not found in axis"

In [16]:
print(f'We have removed {set(data.columns)-set(data_cleaned.columns)}')

We have removed {'Fuel Oil #2 Use (kBtu)', 'Parent Property Name', 'District Chilled Water Use (kBtu)', 'Water Use (All Water Sources) (kgal)', 'Annual Maximum Demand (kW)', 'Fuel Oil #5 & 6 Use (kBtu)', 'Parent Property Id', 'Propane Use (kBtu)', 'Metered Areas  (Water)', 'District Hot Water Use (kBtu)', 'Address 2 (self-reported)', 'Diesel #2 Use (kBtu)', 'Annual Maximum Demand (MM/YYYY)', 'Water Use Intensity (All Water Sources) (gal/ft²)', '3rd Largest Property Use Type', '3rd Largest Property Use Type - Gross Floor Area (ft²)', '2nd Largest Property Use Type', 'District Steam Use (kBtu)', 'Fuel Oil #4 Use (kBtu)', 'Fuel Oil #1 Use (kBtu)', '2nd Largest Property Use - Gross Floor Area (ft²)'}


In [18]:
data_cleaned.columns

Index(['Property Name', 'BBL - 10 digits',
       'NYC Borough, Block and Lot (BBL) self-reported',
       'NYC Building Identification Number (BIN)', 'Address 1 (self-reported)',
       'Postal Code', 'Street Number', 'Street Name', 'Borough',
       'DOF Gross Floor Area (ft²)', 'Self-Reported Gross Floor Area (ft²)',
       'Primary Property Type - Self Selected',
       'List of All Property Use Types at Property',
       'Largest Property Use Type',
       'Largest Property Use Type - Gross Floor Area (ft²)', 'Year Built',
       'Number of Buildings', 'Occupancy', 'Metered Areas (Energy)',
       'ENERGY STAR Score', 'Source EUI (kBtu/ft²)',
       'Weather Normalized Source EUI (kBtu/ft²)', 'Site EUI (kBtu/ft²)',
       'Weather Normalized Site EUI (kBtu/ft²)',
       'Weather Normalized Site Electricity Intensity (kWh/ft²)',
       'Weather Normalized Site Natural Gas Intensity (therms/ft²)',
       'Natural Gas Use (kBtu)',
       'Weather Normalized Site Natural Gas Use (ther

In [20]:
data_cleaned.dropna(subset=['ENERGY STAR Score'], inplace=True)

/tmp/ipykernel_12765/2141253730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.dropna(subset=['ENERGY STAR Score'], inplace=True)


In [21]:
data_cleaned

,Property Name,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Postal Code,Street Number,Street Name,Borough,DOF Gross Floor Area (ft²),...,Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Water Required?,Generation Date,DOF Benchmarking Submission Status
0,The Argonaut Building,1.010288e+09,1010287502,1024898,224 West 57th St,10019,Not found,Not found,Manhattan,Not found,...,16672.2,6551394.1,1920103.6,1920103.6,732.4,76.3,656.1,Not found,2018-02-14,Not found
1,Cathedral Preparatory Seminary,4.018720e+09,4-01872-0007,4046340,56-25 92nd Street,11373,Not found,Not found,Queens,Not found,...,23243.7,616343.7,180640.0,184131.9,164.5,109.9,54.6,Not found,2018-02-14,Not found
2,The Nomad Hotel,1.008290e+09,1-00829-0050,1080710,1170 Broadway,10001-7507,Not found,Not found,Manhattan,Not found,...,86776.9,8033914.4,2354605.3,2354605.3,1150.2,438.0,712.3,Not found,2018-02-14,Not found
3,10 West 27 Street Corp,1.008280e+09,1-00828-0053,1015657,1155 Broadway,10001,Not found,Not found,Manhattan,Not found,...,20520.9,1976691.9,579335.2,582516.1,273.4,98.2,175.3,Not found,2018-02-14,Not found
4,Westbury Realty,1.000650e+09,1-00065-0024,1001105,24 John Street,10038,Not found,Not found,Manhattan,Not found,...,NaN,1022951.6,299809.9,299809.9,90.7,0.0,90.7,Not found,2018-02-14,Not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34348,Delta Management- 8301 Ridge Boulevard,3.060160e+09,3-06016-0006,3152378,8301 Ridge Boulevard,11209,Not found,Not found,Brooklyn,Not found,...,54191.7,496449.4,145501.0,145501.0,312.9,270.8,42.0,Not found,2019-03-02,Not found
34350,Mitchell Hill- 2802 Clarendon Road,3.051940e+09,3-05194-0001,3119966,2802 Clarendon Road,11226,Not found,Not found,Brooklyn,Not found,...,32981.3,316524.4,92768.0,92768.0,187.1,160.2,26.8,Not found,2019-03-07,Not found
34351,Mitchell Hill- 1116 Carroll Street,3.012880e+09,3-01288-0038,3033818,1116 Carroll Street,11225,Not found,Not found,Brooklyn,Not found,...,32420.7,277672.0,81381.0,81381.0,181.2,157.7,23.5,Not found,2019-03-07,Not found
34352,Mitchell Hill- 3033 Brighton 13 Street,3.087160e+09,3-08716-0085,3245653,3033 Brighton 13 Street,11235,Not found,Not found,Brooklyn,Not found,...,35467.9,629677.8,184548.0,184548.0,230.4,177.0,53.3,Not found,2019-03-07,Not found


In [24]:
data_cleaned.drop(columns=['Property Name'], inplace=True)

/tmp/ipykernel_12765/2135814782.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.drop(columns=['Property Name'], inplace=True)


In [28]:
for col in data_cleaned.columns:
    if len(data_cleaned[col].unique())==1:
        print(f'One Value {col}')

One Value Street Number
One Value Street Name
One Value DOF Gross Floor Area (ft²)
One Value Water Required?
One Value DOF Benchmarking Submission Status


In [29]:
data_cleaned.drop(columns=['Water Required?', 'DOF Gross Floor Area (ft²)', 'DOF Benchmarking Submission Status'], inplace=True)

/tmp/ipykernel_12765/1547299696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.drop(columns=['Water Required?', 'DOF Gross Floor Area (ft²)', 'DOF Benchmarking Submission Status'], inplace=True)


In [32]:
data_cleaned.head()

,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Postal Code,Street Number,Street Name,Borough,Self-Reported Gross Floor Area (ft²),Primary Property Type - Self Selected,...,Weather Normalized Site Natural Gas Intensity (therms/ft²),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Generation Date
0,1.010288e+09,1010287502,1024898,224 West 57th St,10019,Not found,Not found,Manhattan,169416,Office,...,0.1,1435754.7,16672.2,6551394.1,1920103.6,1920103.6,732.4,76.3,656.1,2018-02-14
1,4.018720e+09,4-01872-0007,4046340,56-25 92nd Street,11373,Not found,Not found,Queens,94380,K-12 School,...,0.2,2068300.1,23243.7,616343.7,180640.0,184131.9,164.5,109.9,54.6,2018-02-14
2,1.008290e+09,1-00829-0050,1080710,1170 Broadway,10001-7507,Not found,Not found,Manhattan,125000,Hotel,...,0.7,8245445.1,86776.9,8033914.4,2354605.3,2354605.3,1150.2,438.0,712.3,2018-02-14
3,1.008280e+09,1-00828-0053,1015657,1155 Broadway,10001,Not found,Not found,Manhattan,50000,Hotel,...,0.4,1848519.4,20520.9,1976691.9,579335.2,582516.1,273.4,98.2,175.3,2018-02-14
4,1.000650e+09,1-00065-0024,1001105,24 John Street,10038,Not found,Not found,Manhattan,50000,Hotel,...,NaN,NaN,NaN,1022951.6,299809.9,299809.9,90.7,0.0,90.7,2018-02-14


In [43]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25454 entries, 0 to 34353
Data columns (total 33 columns):
 #   Column                                                      Non-Null Count  Dtype         
---  ------                                                      --------------  -----         
 0   BBL - 10 digits                                             25025 non-null  float64       
 1   NYC Borough, Block and Lot (BBL) self-reported              25026 non-null  object        
 2   NYC Building Identification Number (BIN)                    24703 non-null  object        
 3   Address 1 (self-reported)                                   25454 non-null  object        
 4   Postal Code                                                 25454 non-null  object        
 5   Street Number                                               25454 non-null  object        
 6   Street Name                                                 25454 non-null  object        
 7   Borough                    

In [70]:
data_cleaned[data_cleaned['BBL - 10 digits']!=data_cleaned['NYC Borough, Block and Lot (BBL) self-reported'].apply(bbl)]

,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Postal Code,Street Number,Street Name,Borough,Self-Reported Gross Floor Area (ft²),Primary Property Type - Self Selected,...,Weather Normalized Site Natural Gas Intensity (therms/ft²),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Generation Date
39,NaN,NaN,NaN,35 WEST 31ST STREET,10001,Not found,Not found,Manhattan,32500,Office,...,NaN,2346.7,NaN,870169.5,255032.1,NaN,160.7,83.5,77.1,2018-02-20
40,NaN,NaN,NaN,222 West 37th Street,10018,Not found,Not found,Manhattan,39655,Office,...,NaN,NaN,NaN,NaN,NaN,NaN,185.7,77.8,NaN,2018-02-20
48,NaN,NaN,NaN,24-41 31st St,11102,Not found,Not found,Queens,14700,Retail Store,...,0.1,145600.0,1642.9,1051476.0,308170.0,305505.5,101.0,7.7,93.2,2018-02-21
163,2.025460e+09,25460027,NaN,164 Bruckner Blvd,10454,Not found,Not found,Bronx,39614,K-12 School,...,2.0,6881326.6,77814.1,2422656.4,710039.9,724986.0,580.3,365.5,214.8,2018-03-09
165,NaN,NaN,NaN,8201 Bay Parkway,11724,Not found,Not found,Brooklyn,48000,Multifamily Housing,...,0.0,33016.7,360.4,85664.2,25106.7,25969.8,15.5,1.8,13.7,2018-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34084,NaN,NaN,3424564,350 Neptune Ave,11235,Not found,Not found,Brooklyn,66869,Office,...,NaN,NaN,NaN,1621291.2,475173.2,475173.2,137.3,0.0,137.3,2019-02-27
34333,2.027630e+09,2-002763-00215,2006551,770 Faile St.,10474,Not found,Not found,Bronx,29300,Multifamily Housing,...,1.5,4089816.5,43260.2,909215.2,266475.7,264941.1,294.2,217.2,77.0,2019-02-27
34335,3.014700e+09,3-001470-0080,3039446,1661 St johns Pl,11233,Not found,Not found,Brooklyn,39600,Multifamily Housing,...,NaN,NaN,NaN,517823.8,151765.5,150904.6,43.9,0.0,43.9,2019-02-27
34336,3.014700e+09,3-001470-0060,3039445,1677 St Johns Pl,11233,Not found,Not found,Brooklyn,39600,Multifamily Housing,...,NaN,NaN,NaN,406278.4,119073.4,118636.7,34.4,0.0,34.4,2019-02-27


In [69]:
def bbl(x):
    if type(x) == int:
        return x
    elif type(x) == float and math.isnan(x):
        return x
    else:
      return int(re.sub(r"\D","", x))
        

In [54]:
data_cleaned['NYC Borough, Block and Lot (BBL) self-reported'][pd.isnull(data_cleaned['NYC Borough, Block and Lot (BBL) self-reported'])].iloc[0]

nan

In [72]:
data_cleaned['NYC Borough, Block and Lot (BBL) self-reported'] = data_cleaned['NYC Borough, Block and Lot (BBL) self-reported'].apply(bbl)

/tmp/ipykernel_12765/4010085036.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['NYC Borough, Block and Lot (BBL) self-reported'] = data_cleaned['NYC Borough, Block and Lot (BBL) self-reported'].apply(bbl)


In [74]:
data_cleaned[data_cleaned['BBL - 10 digits']!=data_cleaned['NYC Borough, Block and Lot (BBL) self-reported']]

,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Postal Code,Street Number,Street Name,Borough,Self-Reported Gross Floor Area (ft²),Primary Property Type - Self Selected,...,Weather Normalized Site Natural Gas Intensity (therms/ft²),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Generation Date
39,NaN,NaN,NaN,35 WEST 31ST STREET,10001,Not found,Not found,Manhattan,32500,Office,...,NaN,2346.7,NaN,870169.5,255032.1,NaN,160.7,83.5,77.1,2018-02-20
40,NaN,NaN,NaN,222 West 37th Street,10018,Not found,Not found,Manhattan,39655,Office,...,NaN,NaN,NaN,NaN,NaN,NaN,185.7,77.8,NaN,2018-02-20
48,NaN,NaN,NaN,24-41 31st St,11102,Not found,Not found,Queens,14700,Retail Store,...,0.1,145600.0,1642.9,1051476.0,308170.0,305505.5,101.0,7.7,93.2,2018-02-21
163,2.025460e+09,2.546003e+07,NaN,164 Bruckner Blvd,10454,Not found,Not found,Bronx,39614,K-12 School,...,2.0,6881326.6,77814.1,2422656.4,710039.9,724986.0,580.3,365.5,214.8,2018-03-09
165,NaN,NaN,NaN,8201 Bay Parkway,11724,Not found,Not found,Brooklyn,48000,Multifamily Housing,...,0.0,33016.7,360.4,85664.2,25106.7,25969.8,15.5,1.8,13.7,2018-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34084,NaN,NaN,3424564,350 Neptune Ave,11235,Not found,Not found,Brooklyn,66869,Office,...,NaN,NaN,NaN,1621291.2,475173.2,475173.2,137.3,0.0,137.3,2019-02-27
34333,2.027630e+09,2.002763e+11,2006551,770 Faile St.,10474,Not found,Not found,Bronx,29300,Multifamily Housing,...,1.5,4089816.5,43260.2,909215.2,266475.7,264941.1,294.2,217.2,77.0,2019-02-27
34335,3.014700e+09,3.001470e+10,3039446,1661 St johns Pl,11233,Not found,Not found,Brooklyn,39600,Multifamily Housing,...,NaN,NaN,NaN,517823.8,151765.5,150904.6,43.9,0.0,43.9,2019-02-27
34336,3.014700e+09,3.001470e+10,3039445,1677 St Johns Pl,11233,Not found,Not found,Brooklyn,39600,Multifamily Housing,...,NaN,NaN,NaN,406278.4,119073.4,118636.7,34.4,0.0,34.4,2019-02-27


In [75]:
data_cleaned.drop(columns=['NYC Borough, Block and Lot (BBL) self-reported'], inplace=True)

/tmp/ipykernel_12765/911747378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.drop(columns=['NYC Borough, Block and Lot (BBL) self-reported'], inplace=True)


In [76]:
data_cleaned.drop(columns=['NYC Building Identification Number (BIN)'], inplace=True)

/tmp/ipykernel_12765/2521623025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.drop(columns=['NYC Building Identification Number (BIN)'], inplace=True)


In [78]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25454 entries, 0 to 34353
Data columns (total 31 columns):
 #   Column                                                      Non-Null Count  Dtype         
---  ------                                                      --------------  -----         
 0   BBL - 10 digits                                             25025 non-null  float64       
 1   Address 1 (self-reported)                                   25454 non-null  object        
 2   Postal Code                                                 25454 non-null  object        
 3   Street Number                                               25454 non-null  object        
 4   Street Name                                                 25454 non-null  object        
 5   Borough                                                     25454 non-null  object        
 6   Self-Reported Gross Floor Area (ft²)                        25454 non-null  int64         
 7   Primary Property Type - Sel

In [79]:
data_cleaned.drop(columns=['Generation Date'], inplace=True)

/tmp/ipykernel_12765/3427773846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.drop(columns=['Generation Date'], inplace=True)


In [87]:
numerical_columns = ['Source EUI (kBtu/ft²)', 
                     'Weather Normalized Source EUI (kBtu/ft²)', 
                     'Site EUI (kBtu/ft²)', 
 'Weather Normalized Site EUI (kBtu/ft²)',
 'Weather Normalized Site Electricity Intensity (kWh/ft²)',
 'Weather Normalized Site Natural Gas Intensity (therms/ft²)',
 'Natural Gas Use (kBtu)',
 'Weather Normalized Site Natural Gas Use (therms)',
 'Electricity Use - Grid Purchase (kBtu)',
 'Electricity Use - Grid Purchase (kWh)',
 'Weather Normalized Site Electricity (kWh)',
 'Total GHG Emissions (Metric Tons CO2e)',
 'Direct GHG Emissions (Metric Tons CO2e)',
 'Indirect GHG Emissions (Metric Tons CO2e)',
                    'Largest Property Use Type - Gross Floor Area (ft²)',
                     'Self-Reported Gross Floor Area (ft²)',
                    'Occupancy',
                    'Number of Buildings',
                    'Age']


In [91]:
data_cleaned['Age'] = 2023 - data_cleaned['Year Built']

/tmp/ipykernel_12765/2295365165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Age'] = 2023 - data_cleaned['Year Built']


In [92]:
data_cleaned.drop(columns=['Year Built'], inplace=True)

/tmp/ipykernel_12765/1433552663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.drop(columns=['Year Built'], inplace=True)


In [97]:
data_cleaned['Self-Reported Gross Floor Area (ft²)']

0        169416
1         94380
2        125000
3         50000
4         50000
          ...  
34348     51960
34350     32800
34351     32350
34352     33800
34353    263259
Name: Self-Reported Gross Floor Area (ft²), Length: 25454, dtype: int64

In [99]:
categorical_variables = ['Metered Areas (Energy)', 
                         'Primary Property Type - Self Selected',
                         'List of All Property Use Types at Property', 
                         'Largest Property Use Type',
                        'Borough',
                        'Postal Code',
                        'Address 1 (self-reported)',
                        'Street Number',
                        'Street Name']